In [2]:
import torch 
# from alphagen.config import *
# from alphagen.data.tokens import *
from alphagen_generic.features import *
from alphagen.data.expression import *
from gan.utils.data import get_data_by_year

In [3]:


instruments: str = "csi300"
from typing import Tuple
import json

def load_alpha_pool(raw) -> Tuple[List[Expression], List[float]]:
    exprs_raw = raw['exprs']
    exprs = [eval(expr_raw.replace('open', 'open_').replace('$', '')) for expr_raw in exprs_raw]
    weights = raw['weights']
    return exprs, weights

def load_alpha_pool_by_path(path: str) -> Tuple[List[Expression], List[float]]:
    with open(path, encoding='utf-8') as f:
        raw = json.load(f)
        return load_alpha_pool(raw)
    
import os
def load_ppo_path(path,name_prefix):
    
    files = os.listdir(path)
    folder = [i for i in files if name_prefix in i][-1]
    names = [i for i in os.listdir(f"{path}/{folder}") if '.json' in i]
    name = sorted(names,key = lambda x:int(x.split('_')[0]))[-1]
    return f"{path}/{folder}/{name}"

# Infer

In [ ]:
from alphagen_qlib.calculator import QLibStockDataCalculator
result = []
paths = os.listdir('out_dso')
name = 'test1'
freq = 'day'
for instruments in ['csi300','csi500']:
    for num in [1,10,20,50,100]:
        for seed in range(5):
            for train_end in range(2016,2021):
                returned = get_data_by_year(
                    train_start = 2010,train_end=train_end,valid_year=train_end+1,test_year =train_end+2,
                    instruments=instruments, target=target,freq=freq,
                )
                data_all, data,data_valid,data_valid_withhead,data_test,data_test_withhead,_ = returned
                
                path = f"out_dso/{name}_{instruments}_{num}_{train_end}_{seed}/pool.json"
                dirname = os.path.dirname(path)
                print(path)
                exprs,weights = load_alpha_pool_by_path(path)
                
                # calculator_test = QLibStockDataCalculator(data_test, target)
                calculator_test = QLibStockDataCalculator(data_all, target)

                ensemble_value = calculator_test.make_ensemble_alpha(exprs, weights)
                ensemble_value = ensemble_value[-data_test.n_days:]
                
                torch.save(ensemble_value.cpu(),f"{dirname}/{train_end}_{num}_{seed}.pt")

# Read Experiment Result and Calculate Metrics

In [ ]:


from alphagen.utils.correlation import batch_pearsonr,batch_spearmanr
device = 'cuda:0'
name = 'test1'
instruments = 'csi300'
result = []
for seed in range(5):
    cur_seed_ic = []
    cur_seed_ric = []
    
    for num in [1,10,20,50,100]:
        for train_end in range(2016,2021):
            returned = get_data_by_year(
                train_start = 2010,train_end=train_end,valid_year=train_end+1,test_year =train_end+2,
                instruments=instruments, target=target,freq=freq,
            )
            data_all, data,data_valid,data_valid_withhead,data_test,data_test_withhead,_ = returned

            dirname = f"out_dso/{name}_{instruments}_{num}_{train_end}_{seed}"
            
            pred = torch.load(f"{dirname}/{train_end}_{num}_{seed}.pt").to('cuda:0')
            tgt = target.evaluate(data_test)
            tgt = target.evaluate(data_all)[-data_test.n_days:,:]

            ic_s = torch.nan_to_num(batch_pearsonr(pred,tgt),nan=0)
            rank_ic_s = torch.nan_to_num(batch_spearmanr(pred,tgt),nan=0)

            cur_seed_ic.append(ic_s)
            cur_seed_ric.append(rank_ic_s)
        ic = torch.cat(cur_seed_ic)
        rank_ic = torch.cat(cur_seed_ric)
        
        ic_mean = ic.mean().item()
        rank_ic_mean = rank_ic.mean().item()
        ic_std = ic.std().item()
        rank_ic_std = rank_ic.std().item()
        tmp = dict(
            seed = seed,
            num = num,
            ic = ic_mean,
            ric = rank_ic_mean,
            icir = ic_mean/ic_std,
            ricir = rank_ic_mean/rank_ic_std,
        )
        result.append(tmp)

import pandas as pd
exp_result = pd.DataFrame(result).groupby(['num','seed']).mean().groupby('num').agg(['mean','std'])
print(exp_result)
            